<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color:skyblue">
TECHNO - ECONOMIC PARAMETERS DATA PROCESSING
<br>
POWER PLANTS
    </div>
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    Main Formatting Notebook
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Each part of the following script was used to proccess the raw data for power plants units of the Dispa-SET_Unleash project.
    <br>
    Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:Skyblue">
    Importing needed libraries
</div>

In [1]:
import os
import csv
from datetime import datetime
import requests
import pandas as pd
from shutil import move
import numpy as np
import shutil
from bs4 import BeautifulSoup

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    2. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory. 
<br>
    If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    2.1. Back Up Directory
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Saving the original files into a Back up folder.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Since in the next steps of the processing data new subfolders and files are going to be created, the original ones are saved in a back up foldet to return them as its default content ones the process will be finished.
</div>

In [3]:
# Additional string to be appended
additional_path = "/RawData/PowerPlants/"
additional_path_6 = "/RawData/PowerPlants_backup/"

# Construct the power_plants_raw_data_folder_path variable
power_plants_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path

print("power_plants_raw_data_folder_path:", power_plants_raw_data_folder_path)

power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/


In [4]:
# Construct the backup_folder_path variable
backup_folder_path = dispaSET_unleash_folder_path + additional_path_6

print("backup_folder_path:", backup_folder_path)

# Create a backup of the directory
if os.path.exists(backup_folder_path):
    shutil.rmtree(backup_folder_path)  # Remove any existing backup if necessary
shutil.copytree(power_plants_raw_data_folder_path, backup_folder_path)

print(f"Backup created at {backup_folder_path}")

backup_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants_backup/
Backup created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants_backup/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    3. Zone(s) Creation
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Entering the zone name or names (in case of more than one zone wanted to be modelled) to create the folder where all data related to the corresponding zone are going to be storage
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
    For European country names use the ISO 3166-1 standars i.e. AT, BE, BG, CH.... etc. to give the zone_name.
<br>
    For non European countries it would rather to call the zone_name with the same word of how it is defined in the data to be downloaded and processed. e.g. 
</div>
    <div style="text-align: justify; margin-left: 4.00em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman; color:skyblue">
        If it is downloading a csv file with all power plants of Spain but just the units fo Pamplona city are wanted, and in the corresponding donwloaded file, Pamplona is refering with the acronim "PMPLN"; set the zone_name variable with the word "PMPLN".
</div>

In [5]:
# List of folder names to create
zone_names = [
"AT",
"BE",
"BG",
"CH",
"CY",
"CZ",
"DE",
"DK",
"EE",
"EL",
"ES",
"FI",
"FR",
"HR",
"HU",
"IE",
"IT",
"LT",
"LU",
"LV",
"MT",
"NL",
"NO",
"PL",
"PT",
"RO",
"SE",
"SI",
"SK",
"UK"
]

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Creating the zones (Countries modelled in Dispa-SET).
</div>

In [6]:
# Dictionary to store created zone paths
created_zones = {}

# Create the zone
for zone_name in zone_names:
    zone_path = os.path.join(power_plants_raw_data_folder_path, zone_name)
    os.makedirs(zone_path, exist_ok=True)
    created_zones[zone_name] = zone_path
    print(f"Created zone: {zone_path}")

# Print the created zone paths
print("Created zones:")
for zone_name, zone_path in created_zones.items():
    print(f"{zone_name}: {zone_path}")
    
created_zones

Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR
Created zone: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU
Created zone: /home/ray/D

{'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT',
 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE',
 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG',
 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH',
 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY',
 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ',
 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE',
 'DK': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK',
 'EE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE',
 'EL': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL',
 'ES': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES',
 'FI': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI',
 'FR': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR',
 'HR': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR',
 'HU': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU',
 'IE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/IE',
 'IT': '/home/ray/Dispa-

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [7]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"power_plants_raw_data_folder_path:                         {power_plants_raw_data_folder_path}")
print (f"zone_names:                                                {zone_names}")
print (f"created_zones:                                             {created_zones}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
power_plants_raw_data_folder_path:                         /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/
zone_names:                                                ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']
created_zones:                                             {'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT', 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE', 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG', 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH', 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY', 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ', 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE', 'DK': '/

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    4. Raw Data Sources Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Entering all the files paths where the raw data is content.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
If a set of files where all the power plants raw data is content, put them all in the local folder which path is:
</div>
    <div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
/Local/Dispa_Set/Path/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/
        </div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
     Order them adding a number between pharentesys (), this ordernation is going to be saved by the variable sources_list. 
</div>
 <div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
        That list is going to be saved to be used as input for next stages.
</div>
    <div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
        Notice that to process the data all the source files have to be .csv files.
</div>
    <div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    In the other hand, it is important to define which zone is refering the file source.
</div>
    <div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
    If all the source file contents data that belongs to only one zone, epecify it in the variable download_links_zone_related applying the same order than the variable sources_list.
    <br>
    If the source file contents data that refers different zones at the same time, specify it with the word "General" in the variable download_links_zone_related using the same order than the variable sources_list.
</div>
    <div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Remember that the next filtering stages depend on the correct setting of this step.
</div>
    <div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Additionally indicate the year of all data is referring to.
</div>
    <div style="text-align: justify; margin-left: 1.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    This is going to be used as the name root under which all next files are going to be created.
</div>

In [8]:
# Year to which data refers to:
data_year = '2016'

In [9]:
# Additional string to be appended
#additional_path_1 = "/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/"

# Construct the power_plants_raw_data_folder_path variable
#power_plants_raw_data_sources_folder_path = dispaSET_unleash_folder_path + additional_path_1
#print("power_plants_raw_data_sources_folder_path:", power_plants_raw_data_sources_folder_path)


additional_path_1 = "/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/"

# Create the power_plants_raw_data_sources_folder_path
power_plants_raw_data_sources_folder_path = dispaSET_unleash_folder_path + additional_path_1 + str(data_year) + "/"
print("power_plants_raw_data_sources_folder_path:", power_plants_raw_data_sources_folder_path)

power_plants_raw_data_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/


In [10]:
#power_plants_raw_data_sources_folder_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_ENTSOE_Raw_Data/"

# List to store the file paths
sources_list = []

# Function to extract the number from within parentheses
def extract_number(filename):
    start = filename.rfind('(') + 1
    end = filename.rfind(')')
    return int(filename[start:end])

# Iterate over all files in the folder
for root, dirs, files in os.walk(power_plants_raw_data_sources_folder_path):
    for file in files:
        # Construct the full file path
        file_path = os.path.join(root, file)
        # Append the file path to the source list
        sources_list.append(file_path)

# Sort the source list based on the numbers within parentheses
sources_list.sort(key=lambda x: extract_number(os.path.basename(x)))

# Print the sorted list of file paths
print(sources_list)

sources_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(0).csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(1).csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(2).csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(3).csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(4).csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(5).csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPla

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(0).csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(1).csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(2).csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(3).csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(4).csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/2016/Installed Capacity Per Production Unit_201601010000-201701010000(5).csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

In [11]:
# List of the download links:
download_links = sources_list

In [12]:
# List of zones related to the download links:
download_links_zone_related = [
"AT",
"BE",
"BG",
"CH",
"CY",
"CZ",
"DE",
"DE",
"DE",
"DE",
"DK",
"EE",
"EL",
"ES",
"FI",
"FR",
"HR",
"HU",
"IE",
"IT",
"LT",
"LU",
"LV",
"MT",
"NL",
"NO",
"PL",
"PT",
"RO",
"SE",
"SI",
"SK",
"UK",
"UK"
]

In [13]:
def save_download_links_to_csv(links, zones, folder_path, data_year):
    # Create the filename using the data year, current date, and time
    now = datetime.now()
    timestamp = now.strftime("%Y%m%d_%H%M%S")
    file_name = f"{data_year}_power_plants_raw_data_sources_{timestamp}.csv"
    
    # Create a folder with the same name as the file (without extension)
    folder_name = os.path.splitext(file_name)[0]
    folder_path = os.path.join(folder_path, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    
    # Combine the folder path and filename
    file_path = os.path.join(folder_path, file_name)
    
    # Write links to CSV file
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write the header
        writer.writerow(['Download_Link_Sources', 'Zone', 'File_Name'])
        
        # Write the links, zones, and file names
        for i, (link, zone) in enumerate(zip(links, zones), start=1):
            writer.writerow([link, zone, i])
    
    print(f"Download links saved to: {file_path}")
    
    return file_path, file_name

# Save the download links to a CSV file and get the file path and name
power_plants_raw_data_sources_file_path, power_plants_raw_data_sources_file_name = save_download_links_to_csv(download_links, download_links_zone_related, power_plants_raw_data_folder_path, data_year)

print("File path:", power_plants_raw_data_sources_file_path)
print("File name:", power_plants_raw_data_sources_file_name)

Download links saved to: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/2016_power_plants_raw_data_sources_20241004_182838/2016_power_plants_raw_data_sources_20241004_182838.csv
File path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/2016_power_plants_raw_data_sources_20241004_182838/2016_power_plants_raw_data_sources_20241004_182838.csv
File name: 2016_power_plants_raw_data_sources_20241004_182838.csv


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [14]:
print (f"dispaSET_unleash_folder_name:                            {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                            {dispaSET_unleash_folder_path}")
print (f"power_plants_raw_data_folder_path:                       {power_plants_raw_data_folder_path}")
print (f"zone_names:                                              {zone_names}")
print (f"created_zones:                                           {created_zones}")
print (f"power_plants_raw_data_sources_file_path:                 {power_plants_raw_data_sources_file_path}")
print (f"power_plants_raw_data_sources_file_name:                 {power_plants_raw_data_sources_file_name}")
print (f"data_year:                                               {data_year}")
print (f"download_links_zone_related:                             {download_links_zone_related}")

dispaSET_unleash_folder_name:                            Dispa-SET_Unleash
dispaSET_unleash_folder_path:                            /home/ray/Dispa-SET_Unleash
power_plants_raw_data_folder_path:                       /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/
zone_names:                                              ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']
created_zones:                                           {'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT', 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE', 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG', 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH', 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY', 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ', 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE', 'DK': '/home/ray/D

In [15]:
created_zones

{'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT',
 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE',
 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG',
 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH',
 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY',
 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ',
 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE',
 'DK': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK',
 'EE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE',
 'EL': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL',
 'ES': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES',
 'FI': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI',
 'FR': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR',
 'HR': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR',
 'HU': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU',
 'IE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/IE',
 'IT': '/home/ray/Dispa-

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    5. Power Plants Raw Data Download Files
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Using the download list given previously to download and save all the power units raw data files inside a folder called as is it specified in the variable power_plants_raw_data_sources_folder_path.
    <br>
    All the downloaded files are named under the ordering of the download_links list.
    <br>
    Additionally, as some files use different kind of column delimiters e.g. ","; ";"; "/"; etc. the code verifies and changes this to be acceptable for the next steps 
</div>

In [16]:
#power_plants_raw_data_sources_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/2023_power_plants_raw_data_sources_20240418_185453/2023_power_plants_raw_data_sources_20240418_185453.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(power_plants_raw_data_sources_file_path)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    download_link = row['Download_Link_Sources']
    file_name = str(row['File_Name'])  # Convert to string
    
    # Check if the download link is valid
    if pd.notnull(download_link) and pd.notnull(file_name):
        # Construct the destination file path
        destination_path = os.path.join(os.path.dirname(power_plants_raw_data_sources_file_path), file_name)
        
        # Copy the content from the download link to the destination file path
        shutil.copy(download_link, destination_path)
        print(f"File '{file_name}' copied successfully.")
    else:
        print("Download link or file name is missing.")


File '1' copied successfully.
File '2' copied successfully.
File '3' copied successfully.
File '4' copied successfully.
File '5' copied successfully.
File '6' copied successfully.
File '7' copied successfully.
File '8' copied successfully.
File '9' copied successfully.
File '10' copied successfully.
File '11' copied successfully.
File '12' copied successfully.
File '13' copied successfully.
File '14' copied successfully.
File '15' copied successfully.
File '16' copied successfully.
File '17' copied successfully.
File '18' copied successfully.
File '19' copied successfully.
File '20' copied successfully.
File '21' copied successfully.
File '22' copied successfully.
File '23' copied successfully.
File '24' copied successfully.
File '25' copied successfully.
File '26' copied successfully.
File '27' copied successfully.
File '28' copied successfully.
File '29' copied successfully.
File '30' copied successfully.
File '31' copied successfully.
File '32' copied successfully.
File '33' copied 

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    5.2. Aditional Power Plants Raw Data
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Adding data of separate and syngle power units that are not included into the full downloaded raw data files.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Introduce the 'Name', 'Power Capacity' and the 'Technology' of the units in the respective list.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    For the 'Techology' field match the current value of the added unit according with the following equivalences:
</div>
<div style="text-align: justify; margin-left: 5.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
    Biomass
<br>
    Fossil Brown coal/Lignite
<br>
    Fossil Coal-derived gas
<br>
    Fossil Gas
<br>
    Fossil Hard coal
<br>
    Fossil Oil
<br>
    Fossil Oil shale
<br>
    Fossil Peat
<br>
    Geothermal
<br>
    Hydro Pumped Storage
<br>
    Hydro Run-of-river and poundage
<br>
    Hydro Water Reservoir
<br>
    Marine
<br>
    Nuclear
<br>
    Other renewable
<br>
    Solar
<br>
    Waste
<br>
    Wind Offshore
<br>
    Wind Onshore
<br>
    Other non-renewable
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Additionally specify the European country acronym modelled by Dispa-SET which the corresponding unit belongs:
</div>

In [17]:
name = ['Kristal _ Solar _ Park']
power_capacity = [99]
technology = ['Solar']
country = ['BE']

In [18]:
# Read the power plants raw data sources file
raw_data_sources = pd.read_csv(power_plants_raw_data_sources_file_path)

# Process each entry in the lists
for i, zone in enumerate(country):
    # Find the first occurrence of the zone in the raw data sources file
    match = raw_data_sources[raw_data_sources['Zone'] == zone].iloc[0]
    file_name = str(match['File_Name'])  # Ensure file_name is a string
    
    # Construct the path to the corresponding CSV file
    csv_file_path = os.path.join(os.path.dirname(power_plants_raw_data_sources_file_path), file_name)
    
    # Read the corresponding CSV file
    df = pd.read_csv(csv_file_path)
    
    # Create a new DataFrame with the new row
    new_row = pd.DataFrame({
        'Name': [name[i]],
        'Current Installed Capacity [MW]': [power_capacity[i]],
        'Intalled Capacity at the beginning of year [MW]': [power_capacity[i]],
        'Production Type': [technology[i]]
    })
    
    # Concatenate the new row to the DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    
    # Save the updated CSV file
    df.to_csv(csv_file_path, index=False)

print("CSV files updated successfully.")

CSV files updated successfully.


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    6. Zone Classification
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Filtering the data contained in each downloaded file accordingly the zone previously specified. 
</div>
<div style="text-align: jusitfy; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    6.1. Zone Definition
</div>
<div style="text-align: jusitfy; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Adding a new column under the name "Country" to each downloaded file which have been relationated with a key Zone in the list download_links_zone_related fulfilled previously
    <br>
    For all those files that have been set with the key "General", it is assumed that so file contains data from various zones, so It will be filtered in a different way.
</div>

In [19]:
# Read the CSV file specified in power_plants_raw_data_sources_file_path
df_sources = pd.read_csv(power_plants_raw_data_sources_file_path)

# Iterate over each row in the DataFrame
for index, row in df_sources.iterrows():
    file_name = str(row['File_Name'])  # Convert to string
    zone = row['Zone']
    
    # Check if the zone is not "General"
    if zone != "General":
        # Construct the path to the corresponding CSV file
        csv_file_path = os.path.join(os.path.dirname(power_plants_raw_data_sources_file_path), file_name)
        
        # Check if the CSV file exists
        if os.path.exists(csv_file_path):
            # Read the CSV file
            df_csv = pd.read_csv(csv_file_path)
            
            # Add a new column "Country" with the value from the "Zone" column
            df_csv['Country'] = zone
            
            # Write the updated DataFrame back to the CSV file
            df_csv.to_csv(csv_file_path, index=False)
            
            print(f"Added 'Country' column to {file_name} with value '{zone}'")
        else:
            print(f"CSV file {file_name} does not exist.")
    else:
        print(f"No action needed for {file_name}")


Added 'Country' column to 1 with value 'AT'
Added 'Country' column to 2 with value 'BE'
Added 'Country' column to 3 with value 'BG'
Added 'Country' column to 4 with value 'CH'
Added 'Country' column to 5 with value 'CY'
Added 'Country' column to 6 with value 'CZ'
Added 'Country' column to 7 with value 'DE'
Added 'Country' column to 8 with value 'DE'
Added 'Country' column to 9 with value 'DE'
Added 'Country' column to 10 with value 'DE'
Added 'Country' column to 11 with value 'DK'
Added 'Country' column to 12 with value 'EE'
Added 'Country' column to 13 with value 'EL'
Added 'Country' column to 14 with value 'ES'
Added 'Country' column to 15 with value 'FI'
Added 'Country' column to 16 with value 'FR'
Added 'Country' column to 17 with value 'HR'
Added 'Country' column to 18 with value 'HU'
Added 'Country' column to 19 with value 'IE'
Added 'Country' column to 20 with value 'IT'
Added 'Country' column to 21 with value 'LT'
Added 'Country' column to 22 with value 'LU'
Added 'Country' col

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [20]:
print (f"dispaSET_unleash_folder_name: {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path: {dispaSET_unleash_folder_path}")
print (f"power_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}")
print (f"zone_names: {zone_names}")
print (f"created_zones: {created_zones}")
print (f"power_plants_raw_data_sources_file_path: {power_plants_raw_data_sources_file_path}")
print (f"power_plants_raw_data_sources_file_name: {power_plants_raw_data_sources_file_name}")
print (f"data_year: {data_year}")
print (f"download_links_zone_related: {download_links_zone_related}")

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/
zone_names: ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']
created_zones: {'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT', 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE', 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG', 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH', 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY', 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ', 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE', 'DK': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK', 'EE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE', 'EL': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL', '

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    6.2. Raw Data File Zone Classification
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Moving each downloaded file to its corresponding folder zone accordinlgy the download_links_zone_related list.
    <br>
    The files relationed to the key "General" just will keep their current location.
</div>

In [21]:
# Read the raw data sources file
df_sources = pd.read_csv(power_plants_raw_data_sources_file_path)

# Create a new column Final_File_Path
df_sources['Final_File_Path'] = ''

# Iterate over each row in the DataFrame
for index, row in df_sources.iterrows():
    file_name = str(row['File_Name'])
    file_path = os.path.join(os.path.dirname(power_plants_raw_data_sources_file_path), file_name)
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Open and read the file
        df_csv = pd.read_csv(file_path)
        
        # Check if the file has the header 'Country'
        if 'Country' in df_csv.columns:
            # Get the corresponding value of 'Zone'
            zone_value = row['Zone']
            
            # Check if the zone folder exists
            if zone_value in zone_names:
                # Construct the destination folder path
                destination_folder = os.path.join(power_plants_raw_data_folder_path, zone_value)
                
                # Move the file to the destination folder
                destination_file_path = os.path.join(destination_folder, file_name)
                move(file_path, destination_file_path)
                print(f"Moved file '{file_name}' to '{destination_folder}'")
                
                # Get the current path of the moved file
                final_file_path = os.path.abspath(destination_file_path)
                df_sources.at[index, 'Final_File_Path'] = final_file_path
            else:
                print(f"Destination folder for zone '{zone_value}' does not exist.")
        else:
            print(f"No 'Country' header found in file '{file_name}'. No action needed.")
    else:
        print(f"File '{file_name}' does not exist.")
        
    # If file was not moved, update Final_File_Path with current path
    if not df_sources.at[index, 'Final_File_Path']:
        df_sources.at[index, 'Final_File_Path'] = os.path.abspath(file_path)

# Save the DataFrame back to the CSV file
df_sources.to_csv(power_plants_raw_data_sources_file_path, index=False)

Moved file '1' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT'
Moved file '2' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE'
Moved file '3' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG'
Moved file '4' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH'
Moved file '5' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY'
Moved file '6' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ'
Moved file '7' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE'
Moved file '8' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE'
Moved file '9' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE'
Moved file '10' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE'
Moved file '11' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK'
Moved file '12' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE'
Moved file '13' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL'
Moved file '14' to '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES'
M

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Extracting a list of all the path of the power plants raw data files.
    <br>
    This list is goning to be used as reference for future filtration stepts.
</div>

In [22]:
# Read the CSV file
df = pd.read_csv(power_plants_raw_data_sources_file_path)

# Create the list power_plants_raw_data_file_list
power_plants_raw_data_file_list = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Convert integer values to string for File_Name and Final_File_Path
    file_name = str(row['File_Name'])
    final_file_path = str(row['Final_File_Path'])
    
    # Concatenate File_Name and Final_File_Path and append to the list
    file_path = os.path.join(file_name, final_file_path)
    power_plants_raw_data_file_list.append(file_path)

# Print the list
print(power_plants_raw_data_file_list)

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/1', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/3', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/4', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/5', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/6', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/7', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/8', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/9', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/10', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/11', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/12', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/13', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/14', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/15', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/16', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/17', '/home/ray/Dispa-SET_Unleash/RawData/Po

In [23]:
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Extract the folder path
    folder_path = os.path.dirname(row['Final_File_Path'])
    
    # Update the DataFrame with the folder path
    df.at[index, 'Folder_Path'] = folder_path

# Save the DataFrame back to the CSV file
df.to_csv(power_plants_raw_data_sources_file_path, index=False)

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [24]:
print (f"dispaSET_unleash_folder_name: {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path: {dispaSET_unleash_folder_path}")
print (f"power_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}")
print (f"zone_names: {zone_names}")
print (f"created_zones: {created_zones}")
print (f"power_plants_raw_data_sources_file_path: {power_plants_raw_data_sources_file_path}")
print (f"power_plants_raw_data_sources_file_name: {power_plants_raw_data_sources_file_name}")
print (f"data_year: {data_year}")
print (f"download_links_zone_related: {download_links_zone_related}")
print (f"power_plants_raw_data_file_list: {power_plants_raw_data_file_list}")

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/
zone_names: ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']
created_zones: {'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT', 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE', 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG', 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH', 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY', 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ', 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE', 'DK': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK', 'EE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE', 'EL': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL', '

<div style="text-align: jusitfy; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
7. Data Formating
</div>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
7.1. Clean Data File Creation
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Creating for each zone an empty csv file with all the technical features needed for Dispa-SET simulations as headers.
    <br>
This file will be named under the value of the variable data year previously specified.
    <br>
On this csv file all the filtered data in the following steps will be written.
</div>

In [25]:
# Read the CSV file
df = pd.read_csv(power_plants_raw_data_sources_file_path)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Get the folder path and file name
    folder_path = row['Folder_Path']
    file_name = f"{data_year}.csv"
    
    # Check if the file already exists in the folder
    file_path = os.path.join(folder_path, file_name)
    if os.path.exists(file_path):
        print(f"File '{file_name}' already exists in '{folder_path}'.")
        # Update the DataFrame with the existing file name and path
        df.at[index, 'Clean_File_Name'] = file_name
        df.at[index, 'Final_Clean_File_Path'] = file_path
    else:
        # Create the new CSV file with the specified headers
        headers = ["", "Unit", "PowerCapacity", "Nunits", "Zone", "Zone_th", "Zone_h2", "Technology", "Fuel", "Efficiency",
                   "MinUpTime", "MinDownTime", "RampUpRate", "RampDownRate", "StartUpCost", "NoLoadCost_pu", "RampingCost",
                   "PartLoadMin", "MinEfficiency", "StartUpTime", "CO2Intensity", "CHPType", "CHPPowerToHeat",
                   "CHPPowerLossFactor", "CHPMaxHeat", "COP", "Tnominal", "coef_COP_a", "coef_COP_b", "STOCapacity",
                   "STOSelfDischarge", "STOMaxChargingPower", "STOChargingEfficiency", "WaterWithdrawal", "WaterConsumption", "Status", "Source",
                   "Company", "Lat", "Lon"]
        with open(file_path, 'w') as f:
            f.write(','.join(headers))
        print(f"Created file '{file_name}' in '{folder_path}'.")
        # Update the DataFrame with the new file name and path
        df.at[index, 'Clean_File_Name'] = file_name
        df.at[index, 'Final_Clean_File_Path'] = file_path

# Create the list of clean data file paths
power_plants_clean_data_file_list = df['Final_Clean_File_Path'].tolist()

# Remove the header from the list
power_plants_clean_data_file_list.pop(0)

# Save the DataFrame back to the CSV file
df.to_csv(power_plants_raw_data_sources_file_path, index=False)

Created file '2016.csv' in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT'.
Created file '2016.csv' in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE'.
Created file '2016.csv' in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG'.
Created file '2016.csv' in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH'.
Created file '2016.csv' in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY'.
Created file '2016.csv' in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ'.
Created file '2016.csv' in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE'.
File '2016.csv' already exists in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE'.
File '2016.csv' already exists in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE'.
File '2016.csv' already exists in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE'.
Created file '2016.csv' in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK'.
Created file '2016.csv' in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE'.
Created

In [26]:
# Read the CSV file
df = pd.read_csv(power_plants_raw_data_sources_file_path)

# Create the list power_plants_clean_data_file_list
power_plants_clean_data_file_list = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Convert integer values to string for File_Name and Final_Clean_File_Path
    file_name = str(row['File_Name'])
    final_file_path = str(row['Final_Clean_File_Path'])
    
    # Concatenate File_Name and Final_Clean_File_Path and append to the list
    file_path = os.path.join(file_name, final_file_path)
    power_plants_clean_data_file_list.append(file_path)

# Print the list
print(power_plants_clean_data_file_list)

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/20

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [27]:
print (f"dispaSET_unleash_folder_name: {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path: {dispaSET_unleash_folder_path}")
print (f"power_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}")
print (f"zone_names: {zone_names}")
print (f"created_zones: {created_zones}")
print (f"power_plants_raw_data_sources_file_path: {power_plants_raw_data_sources_file_path}")
print (f"power_plants_raw_data_sources_file_name: {power_plants_raw_data_sources_file_name}")
print (f"data_year: {data_year}")
print (f"download_links_zone_related: {download_links_zone_related}")
print (f"power_plants_raw_data_file_list: {power_plants_raw_data_file_list}")
print (f"power_plants_clean_data_file_list: {power_plants_clean_data_file_list}")

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/
zone_names: ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']
created_zones: {'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT', 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE', 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG', 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH', 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY', 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ', 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE', 'DK': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK', 'EE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE', 'EL': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL', '

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.2. Dictionary Files
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Getting the current paths of all the dictionary files that are going to be used as reference for the next filtering data process.
        <br>
    The dictionady files are .csv files that contain a small data base of al the technical denomination ussually used for the power units, e.g. 'PowerCapacity', 'Energy Source", 'Technology' etc.
</div>

In [28]:
#dispaSET_unleash_folder_path = "/home/ray/Dispa-SET_Unleash"  # Replace with your actual path

# List of file names
file_names = [
    "ENTSOE_Production_Type.csv",
    "power_plants_clean_data_equivalent_headers.csv",
    "power_plants_all_data_equivalent_technologies.csv",
    "power_plants_all_data_equivalent_fuels.csv",
    "power_plants_all_data_equivalent_CHPTypes.csv",
    "EU_Power_Units_Technical_Features.csv"
]

# Dictionary to store file paths
file_paths = {}

# Construct file paths
for file_name in file_names:
    # Construct the file path
    file_path = os.path.join(dispaSET_unleash_folder_path, "RawData", "PowerPlants", "Dictionaries", file_name)

    # Store the file path in the dictionary
    file_paths[file_name] = file_path

# Define variables explicitly using the file paths
EU_Power_Units_Technical_Features_file_path = file_paths["EU_Power_Units_Technical_Features.csv"]
power_plants_clean_data_equivalent_headers_file_path = file_paths["power_plants_clean_data_equivalent_headers.csv"]
power_plants_all_data_equivalent_technologies_file_path = file_paths["power_plants_all_data_equivalent_technologies.csv"]
power_plants_all_data_equivalent_fuels_file_path = file_paths["power_plants_all_data_equivalent_fuels.csv"]
power_plants_all_data_equivalent_CHPTypes_file_path = file_paths["power_plants_all_data_equivalent_CHPTypes.csv"]
ENTSOE_Production_Type_file_path = file_paths["ENTSOE_Production_Type.csv"]
# ... Define other variables similarly using file_paths dictionary

# The variables are now accessible throughout your code.

# Print the variables (optional)
for file_name, file_path in file_paths.items():
     print(f"{file_name.split('.')[0]}_file_path: {file_path}")

ENTSOE_Production_Type_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/Dictionaries/ENTSOE_Production_Type.csv
power_plants_clean_data_equivalent_headers_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/Dictionaries/power_plants_clean_data_equivalent_headers.csv
power_plants_all_data_equivalent_technologies_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/Dictionaries/power_plants_all_data_equivalent_technologies.csv
power_plants_all_data_equivalent_fuels_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/Dictionaries/power_plants_all_data_equivalent_fuels.csv
power_plants_all_data_equivalent_CHPTypes_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/Dictionaries/power_plants_all_data_equivalent_CHPTypes.csv
EU_Power_Units_Technical_Features_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/Dictionaries/EU_Power_Units_Technical_Features.csv


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [29]:
print (f"dispaSET_unleash_folder_name: {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path: {dispaSET_unleash_folder_path}")
print (f"power_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}")
print (f"zone_names: {zone_names}")
print (f"created_zones: {created_zones}")
print (f"power_plants_raw_data_sources_file_path: {power_plants_raw_data_sources_file_path}")
print (f"power_plants_raw_data_sources_file_name: {power_plants_raw_data_sources_file_name}")
print (f"data_year: {data_year}")
print (f"download_links_zone_related: {download_links_zone_related}")
print (f"power_plants_raw_data_file_list: {power_plants_raw_data_file_list}")
print (f"power_plants_clean_data_file_list: {power_plants_clean_data_file_list}")
print (f"power_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}")
print (f"power_plants_all_data_equivalent_technologies_file_path: {power_plants_all_data_equivalent_technologies_file_path}")
print (f"power_plants_all_data_equivalent_fuels_file_path: {power_plants_all_data_equivalent_fuels_file_path}")
print (f"power_plants_all_data_equivalent_CHPTypes_file_path: {power_plants_all_data_equivalent_CHPTypes_file_path}")
print (f"EU_Power_Units_Technical_Features_file_path: {EU_Power_Units_Technical_Features_file_path}")
print (f"ENTSOE_Production_Type_file_path: {ENTSOE_Production_Type_file_path}")

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/
zone_names: ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']
created_zones: {'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT', 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE', 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG', 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH', 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY', 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ', 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE', 'DK': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK', 'EE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE', 'EL': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL', '

In [30]:
power_plants_clean_data_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.3. ENTSOE SOURCE FILES DATA PRODUCTION TECHNOLOGY TYPE
</div>
<div style="text-align: Justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Getting the technology and fuel clasification for all the raw data from ENTSO-E source.
</div>
<div style="text-align: Justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Since all the data obtained from ENTSO-E web portal does not make a classification of generation technology and fuel source, it is going to use the dictionary called "ENTSO-E_Production_Type.csv" where all the equivalent technologies and fuels are going to be extracted accoding the "Production Type" classification used by ENTSO-E.
</div>

In [31]:
# Read the CSV files
#power_plants_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/2023_power_plants_raw_data_sources_20240507_132711/2023_power_plants_raw_data_sources_20240507_132711.csv"
#entsoe_production_type_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ENTSOE_Production_Type.csv"

power_plants_df = pd.read_csv(power_plants_raw_data_sources_file_path)
entsoe_production_type_df = pd.read_csv(ENTSOE_Production_Type_file_path)

# Function to process each text file
def process_text_file(text_file_path):
    # Read the text file
    with open(text_file_path, 'r') as file:
        lines = file.readlines()

    # Check if 'Production Type' column exists
    header = lines[0].strip().split(',')
    if 'Production Type' in header:
        # Add 'Technology' and 'Fuel' columns if not present
        if 'Technology' not in header:
            header.extend(['Technology', 'Fuel'])
            lines[0] = ','.join(header) + '\n'

        # Extract 'Production Type' column index
        production_type_index = header.index('Production Type')

        # Process each row in the text file
        for i in range(1, len(lines)):
            fields = lines[i].strip().split(',')
            production_type = fields[production_type_index]

            # Find corresponding technology and fuel
            match_row = entsoe_production_type_df[entsoe_production_type_df['Complete Term'] == production_type]
            if not match_row.empty:
                technology = match_row['technology'].values[0]
                fuel = match_row['fuel'].values[0]
                fields.extend([technology, fuel])
                lines[i] = ','.join(fields) + '\n'

        # Write back to the text file
        with open(text_file_path, 'w') as file:
            file.writelines(lines)
        
        # Print message when the file is done
        print(f"Processed {text_file_path}")

# Process each row in power_plants_df
for index, row in power_plants_df.iterrows():
    text_file_path = row['Final_File_Path']
    if os.path.exists(text_file_path):
        process_text_file(text_file_path)

Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/1
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/3
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/4
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/5
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/6
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/7
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/8
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/9
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/10
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/11
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/12
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/13
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/14
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/15
Processed /home/ray/Dispa-SET_Unleash/RawData/Pow

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Copying all the needed columns from the raw data files to the clean data files in base to the order given by the lists power_plants_raw_data_file_list and power_plants_clean_data_file_list.
        <br>
    This processes is made to each zone calling the function copy_columns_to_clean_data.py
</div>

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [32]:
power_plants_raw_data_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/1',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/3',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/4',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/5',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/6',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/7',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/8',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/9',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/10',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/11',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/12',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/13',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/14',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/15',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/16',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/17',
 '/home/ray/Dispa-SET_U

In [33]:
power_plants_clean_data_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

In [34]:
from copy_columns_to_clean_data import copy_columns_to_clean_data
# Iterate over each pair of files
for raw_data_file, clean_data_file in zip(power_plants_raw_data_file_list, power_plants_clean_data_file_list):
    # Call the function with appropriate arguments
    copy_columns_to_clean_data(raw_data_file, clean_data_file, power_plants_clean_data_equivalent_headers_file_path)

Columns copied successfully.
Columns copied successfully.
Columns copied successfully.
Columns copied successfully.
Columns copied successfully.
Columns copied successfully.
Columns copied successfully.


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/copy_columns_to_clean_data.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clean_data_df = pd.concat([clean_data_df, new_rows], ignore_index=True)
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/copy_columns_to_clean_data.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clean_data_df = pd.concat([clean_data_df, new_rows], ignore_index=True)
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/copy

Columns copied successfully.
Columns copied successfully.
Columns copied successfully.
Columns copied successfully.
Columns copied successfully.
Columns copied successfully.


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/copy_columns_to_clean_data.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clean_data_df = pd.concat([clean_data_df, new_rows], ignore_index=True)
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/copy_columns_to_clean_data.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clean_data_df = pd.concat([clean_data_df, new_rows], ignore_index=True)


Columns copied successfully.
Columns copied successfully.


...

Columns copied successfully.


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/copy_columns_to_clean_data.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clean_data_df = pd.concat([clean_data_df, new_rows], ignore_index=True)
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/copy_columns_to_clean_data.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clean_data_df = pd.concat([clean_data_df, new_rows], ignore_index=True)


Columns copied successfully.
Columns copied successfully.
Columns copied successfully.
Columns copied successfully.


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/copy_columns_to_clean_data.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clean_data_df = pd.concat([clean_data_df, new_rows], ignore_index=True)


Columns copied successfully.


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Readdapting the power_plant_clean_file_list variable.
</div>

In [35]:
# Function to remove duplicates while preserving order
def remove_duplicates_preserve_order(file_list):
    seen = set()
    unique_list = []
    for item in file_list:
        if item not in seen:
            unique_list.append(item)
            seen.add(item)
    return unique_list

# Remove duplicates
unique_power_plants_clean_data_file_list = remove_duplicates_preserve_order(power_plants_clean_data_file_list)

# Print the unique list to verify
print(unique_power_plants_clean_data_file_list)

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/IE/20

In [36]:
power_plants_clean_data_file_list = unique_power_plants_clean_data_file_list
power_plants_clean_data_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.5. Units Name Field Fulfill
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Verifying the "Unit" field in all the rows of each cleand data file.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 1px; font-family: TimesNewRoman;color:skyblue">
     If there is no value, the corresponding latitud and longitud value is going to be used as identifier of the corresponding unit. 
        <br>
    If there is no value in the latitud and longitud fields the corresponding "Company" data is going to be taken.
    <br>
    Finally if there is not any company related data, the "Unit" field is going to be performed form the zone name plus an increasing numner.
        <br>
    This processes is made to each zone calling the function units_name_fulfilling.py
</div>

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [37]:
power_plants_clean_data_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

In [38]:
from Data_Processing_Functions import fill_unit_field

# List of file paths
#power_plants_clean_data_file_list = ['path_to_file1.csv', 'path_to_file2.csv', 'path_to_file3.csv']

# Iterate over each file path
for file_path in power_plants_clean_data_file_list:
    fill_unit_field(file_path)

Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.
Unit field filled successfully.


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.6. Nunits Field Fulfill
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Verifying the "Nunit" field in all the rows of each cleand data file.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
     If there is no value, the field is gonna be filled by 1.
        <br>
    This processes is made for each zone calling the function fill_nunits_column.py
</div>

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [39]:
power_plants_clean_data_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

In [40]:
from Data_Processing_Functions import fill_nunits_column

# List of file paths
#clean_data_file_paths = ["path/to/your/clean_data1.csv", "path/to/your/clean_data2.csv", ...]

# Apply the function to each file
for file_path in power_plants_clean_data_file_list:
    fill_nunits_column(file_path)

Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits column processed successfully.
Nunits colum

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.7. Zone Field Filter
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Verifying the "Zone" field in all the rows of each cleand data file.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
     If there is no value, the field is gonna be took out from the file.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
     Additionally all the eliminated rows are going to be storage in a csv file called power_plants_all_data_not_defined_units.
        <br>
    This processes is made for each zone calling the function units_name_filter.py
</div>

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [41]:
print (f"dispaSET_unleash_folder_name: {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path: {dispaSET_unleash_folder_path}")
print (f"power_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}")
print (f"zone_names: {zone_names}")
print (f"created_zones: {created_zones}")
print (f"power_plants_raw_data_sources_file_path: {power_plants_raw_data_sources_file_path}")
print (f"power_plants_raw_data_sources_file_name: {power_plants_raw_data_sources_file_name}")
print (f"data_year: {data_year}")
print (f"download_links_zone_related: {download_links_zone_related}")
print (f"power_plants_raw_data_file_list: {power_plants_raw_data_file_list}")
print (f"power_plants_clean_data_file_list: {power_plants_clean_data_file_list}")
print (f"power_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}")
print (f"power_plants_all_data_equivalent_technologies_file_path: {power_plants_all_data_equivalent_technologies_file_path}")
print (f"power_plants_all_data_equivalent_fuels_file_path: {power_plants_all_data_equivalent_fuels_file_path}")
print (f"power_plants_all_data_equivalent_CHPTypes_file_path: {power_plants_all_data_equivalent_CHPTypes_file_path}")
print (f"EU_Power_Units_Technical_Features_file_path: {EU_Power_Units_Technical_Features_file_path}")

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/
zone_names: ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']
created_zones: {'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT', 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE', 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG', 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH', 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY', 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ', 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE', 'DK': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK', 'EE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE', 'EL': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL', '

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 16px; font-family: TimesNewRoman;color:skyblue">
    7.7.1. Not Defined Units File Creation
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Creating a .csv file for each zone where all those units that has some missing data in one of their field are going to be written in such as all the units that are took out from the final clean data file for some other reason.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    This is done for tracking data purposes.
</div>

In [42]:
# Function to create the CSV file with specified headers
def create_csv_file(file_path, headers):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)

# Define the headers for the CSV file
headers = ["", "Unit", "PowerCapacity", "Nunits", "Zone", "Zone_th", "Zone_h2", "Technology", "Fuel", "Efficiency",
           "MinUpTime", "MinDownTime", "RampUpRate", "RampDownRate", "StartUpCost", "NoLoadCost_pu", "RampingCost",
           "PartLoadMin", "MinEfficiency", "StartUpTime", "CO2Intensity", "CHPType", "CHPPowerToHeat",
           "CHPPowerLossFactor", "CHPMaxHeat", "COP", "Tnominal", "coef_COP_a", "coef_COP_b", "STOCapacity",
           "STOSelfDischarge", "STOMaxChargingPower", "STOChargingEfficiency", "WaterWithdrawal", "WaterConsumption", "Status", "Source",
           "Company", "Lat", "Lon"]

# List to store the paths of created CSV files
power_plants_all_data_not_defined_units_file_list = []

# Read the CSV file and create CSV files in each folder path
with open(power_plants_raw_data_sources_file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        folder_path = row['Folder_Path']
        file_name = 'power_plants_all_data_not_defined_units.csv'
        file_path = os.path.join(folder_path, file_name)
        create_csv_file(file_path, headers)
        print(f"CSV file created at {file_path}")
        power_plants_all_data_not_defined_units_file_list.append(file_path)

print("All CSV files created successfully.")

CSV file created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/power_plants_all_data_not_defined_units.csv
CSV file created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/power_plants_all_data_not_defined_units.csv
CSV file created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/power_plants_all_data_not_defined_units.csv
CSV file created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/power_plants_all_data_not_defined_units.csv
CSV file created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/power_plants_all_data_not_defined_units.csv
CSV file created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/power_plants_all_data_not_defined_units.csv
CSV file created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv
CSV file created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv
CSV file created at /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_pla

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 16px; font-family: TimesNewRoman;color:skyblue">
    7.7.2. No Data Zone Field Elimination
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Eliminating all the data that does not have a zone, country, region etc. in its Zone field.
</div>

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [43]:
power_plants_all_data_not_defined_units_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv',
 '/home/ra

In [44]:
power_plants_clean_data_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

In [45]:
from Data_Processing_Functions import move_rows_with_empty_zone

# Iterate over each pair of files
for clean_data_file, all_data_file in zip(power_plants_clean_data_file_list, power_plants_all_data_not_defined_units_file_list):
    # Call the function with appropriate arguments
    move_rows_with_empty_zone([clean_data_file], [all_data_file])

Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv
Processed /home/ray/

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.8. Technology Field Filter
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Re-naming the current value of all the fileds of Technology column.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Since the generation technology name of the power units may not be homogeneous among all data sources. The current value of this field is compared with the technology equivalence dictionary database to obtain the one corresponding to the Dispa-SET format.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Additionally all the rows which Technolgy field have not been matched with any equivalency are going to be took out from the clean data file and added to the not defined units data list.
    <br>
    This process is done for each zone.
</div>

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [46]:
print (f"dispaSET_unleash_folder_name:                                {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                                {dispaSET_unleash_folder_path}")
print (f"power_plants_raw_data_folder_path:                           {power_plants_raw_data_folder_path}")
print (f"zone_names:                                                  {zone_names}")
print (f"created_zones:                                               {created_zones}")
print (f"power_plants_raw_data_sources_file_path:                     {power_plants_raw_data_sources_file_path}")
print (f"power_plants_raw_data_sources_file_name:                     {power_plants_raw_data_sources_file_name}")
print (f"data_year:                                                   {data_year}")
print (f"download_links_zone_related:                                 {download_links_zone_related}")
print (f"power_plants_raw_data_file_list:                             {power_plants_raw_data_file_list}")
print (f"power_plants_clean_data_file_list:                           {power_plants_clean_data_file_list}")
print (f"power_plants_clean_data_equivalent_headers_file_path:        {power_plants_clean_data_equivalent_headers_file_path}")
print (f"power_plants_all_data_equivalent_technologies_file_path:     {power_plants_all_data_equivalent_technologies_file_path}")
print (f"power_plants_all_data_equivalent_fuels_file_path:            {power_plants_all_data_equivalent_fuels_file_path}")
print (f"power_plants_all_data_equivalent_CHPTypes_file_path:         {power_plants_all_data_equivalent_CHPTypes_file_path}")
print (f"EU_Power_Units_Technical_Features_file_path:                 {EU_Power_Units_Technical_Features_file_path}")
print (f"power_plants_raw_data_file_list:                             {power_plants_raw_data_file_list}")
print (f"power_plants_all_data_not_defined_units_file_list:           {power_plants_all_data_not_defined_units_file_list}")

dispaSET_unleash_folder_name:                                Dispa-SET_Unleash
dispaSET_unleash_folder_path:                                /home/ray/Dispa-SET_Unleash
power_plants_raw_data_folder_path:                           /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/
zone_names:                                                  ['AT', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK']
created_zones:                                               {'AT': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT', 'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE', 'BG': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG', 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH', 'CY': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY', 'CZ': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ', 'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE'

In [47]:
from Data_Processing_Functions import update_technology_equivalents

# Iterate over each pair of files
for clean_data_file, not_defined_units_file in zip(power_plants_clean_data_file_list, power_plants_all_data_not_defined_units_file_list):
    # Call the function with appropriate arguments
    update_technology_equivalents(clean_data_file, power_plants_all_data_equivalent_technologies_file_path, not_defined_units_file)

Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology equivalents updated successfully.
Technology

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [48]:
power_plants_clean_data_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

In [49]:
power_plants_all_data_not_defined_units_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/power_plants_all_data_not_defined_units.csv',
 '/home/ra

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    7.9. Fuel Field Filter
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Re-naming the current value of all the fields of Fuel column.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Since the fuel sources names of the power units may not be homogeneous among all data sources. The current value of this field is compared with the fuel equivalence dictionary database to obtain the one corresponding to the Dispa-SET format.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Additionally all the rows which Fuel field have not been matched with any equivalency are going to be took out fron the cleand data file and added to the not defined units data list. 
    <br>
    This process is done for each zone.
</div>

In [50]:
from Data_Processing_Functions import update_fuel_equivalents

# Iterate over each pair of files
for clean_data_file_path, not_defined_units_file_path in zip(power_plants_clean_data_file_list, power_plants_all_data_not_defined_units_file_list):
    # Call the function with appropriate arguments
    update_fuel_equivalents(clean_data_file_path, power_plants_all_data_equivalent_fuels_file_path, not_defined_units_file_path)

Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated successfully.
Fuel equivalents updated 

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    7.10. Unit Name Filtering
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Re-naming those units wiht different fueatures e.g. techonology, fuel, but the belongs to the location/central or generation park.
</div>

In [51]:
def modify_duplicate_units(file_path):
  """
  Modifies duplicate unit values in a CSV file.

  Args:
    file_path: The path to the CSV file.
  """

  df = pd.read_csv(file_path)

  for unit in df['Unit'].unique():
    duplicate_rows = df[df['Unit'] == unit]
    if len(duplicate_rows) > 1:
      df.loc[duplicate_rows.index, 'Unit'] = [f"{unit}_{i+1}" for i in range(len(duplicate_rows))]

  df.to_csv(file_path, index=False)
  print(f"Processed file: {file_path}")

def process_files(power_plants_raw_data_sources_file_path):
  """
  Processes CSV files based on information in the specified file.

  Args:
    power_plants_raw_data_sources_file_name: The path to the CSV file containing file paths.
  """

  df = pd.read_csv(power_plants_raw_data_sources_file_path)

  for index, row in df.iterrows():
    file_path = row['Final_Clean_File_Path']
    modify_duplicate_units(file_path)

process_files(power_plants_raw_data_sources_file_path)

Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv
Processed fi

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.11. Total Installed Capacity
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
The last power plant units sources do not take account decentralized renewable generation.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
In order to compense this gab additional sources (Total installed Capacity by each country) are used.
<br>
For this purpose, the requiered data were screaped and saved in the following path:
</div>

In [52]:
# Additional string to be appended
additional_path_3 = "/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/"

# Construct the power_plants_raw_data_folder_path variable
total_installed_capacity_per_production_type_folder_path = dispaSET_unleash_folder_path + additional_path_3
print("total_installed_capacity_per_production_type_folder_path:", total_installed_capacity_per_production_type_folder_path)

total_installed_capacity_per_production_type_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/


In [53]:
def get_file_paths(directory_path):
  """
  Gets a list of file paths within a given directory.

  Args:
    directory_path: The path to the directory.

  Returns:
    A list of file paths.
  """

  file_paths = []
  for root, _, files in os.walk(directory_path):
    for file in files:
      if file.endswith(".csv"):
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
  return file_paths

total_installed_capacity_per_production_type_file_path_list = get_file_paths(total_installed_capacity_per_production_type_folder_path)

total_installed_capacity_per_production_type_file_path_list = sorted(total_installed_capacity_per_production_type_file_path_list, key=lambda x: os.path.basename(x))

total_installed_capacity_per_production_type_file_path_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/AT_installed_capacity.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BE_installed_capacity.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BG_installed_capacity.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CH_installed_capacity.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CY_installed_capacity.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CZ_installed_capacity.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_So

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Eliminating unnecesary rows from the total_installed_capacity_per_production_type files.
</div>

In [54]:
def remove_header_values(file_list):
  """Removes rows where 'Production Type' matches the column header and prints messages.

  Args:
    file_list: A list of CSV file paths.
  """

  for file_path in file_list:
    df = pd.read_csv(file_path)
    original_df = df.copy()

    df = df[df['Production Type'] != 'Production Type']

    if not df.equals(original_df):
      df.to_csv(file_path, index=False)
      print(f"File modified: {file_path}")
    else:
      print(f"No changes made to: {file_path}")

remove_header_values(total_installed_capacity_per_production_type_file_path_list)

File modified: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/AT_installed_capacity.csv
File modified: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BE_installed_capacity.csv
File modified: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BG_installed_capacity.csv
File modified: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CH_installed_capacity.csv
File modified: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CY_installed_capacity.csv
File modified: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CZ_installed_capacity.csv
File modif

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Fullfilling empty fields in the 'total_installed_capacity_per_production_type' files
</div>

In [55]:
def fill_empty_fields(file_path):
    """
    Processes a single CSV file to fill empty fields from the second column to the last column.

    Args:
    - file_path: The path to the CSV file.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Iterate over columns from the second to the last column
    for col_index in range(1, len(df.columns)):
        for row_index in range(len(df)):
            # Check if the field is empty or NaN
            if pd.isna(df.iloc[row_index, col_index]) or df.iloc[row_index, col_index] == '':
                # Look for the nearest previous column with any value
                filled_value = 0  # Default value to fill if no value is found
                found_value = False

                # Check all previous columns for the first non-empty value
                for prev_col_index in range(col_index - 1, -1, -1):
                    prev_value = df.iloc[row_index, prev_col_index]
                    # Check if there is any value (not NaN or empty)
                    if pd.notna(prev_value) and prev_value != '':
                        filled_value = prev_value
                        found_value = True
                        break

                # If a value is found, fill the empty field with it
                # Otherwise, it remains as the default '0'
                df.iloc[row_index, col_index] = filled_value

    # Save the modified DataFrame back to the CSV
    df.to_csv(file_path, index=False)
    print(f"Processed file: {file_path}")

def process_files(file_path_list):
    """
    Processes each CSV file in the list of file paths.

    Args:
    - file_path_list: List of file paths to process.
    """
    for file_path in file_path_list:
        fill_empty_fields(file_path)

process_files(total_installed_capacity_per_production_type_file_path_list)

Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/AT_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BE_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BG_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CH_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CY_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CZ_installed_capacity.csv


/tmp/ipykernel_571606/3320574282.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Fossil Peat' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[row_index, col_index] = filled_value
/tmp/ipykernel_571606/3320574282.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Fossil Peat' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[row_index, col_index] = filled_value
/tmp/ipykernel_571606/3320574282.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Fossil Peat' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[row_index, col_index] = filled_value
/tmp/ipykernel_571606/3320574282.py:31: Future

Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/DE_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/DK_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/EE_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/EL_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/ES_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/FI_installed_capacity.csv
Proc

/tmp/ipykernel_571606/3320574282.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Nuclear' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[row_index, col_index] = filled_value
/tmp/ipykernel_571606/3320574282.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Nuclear' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[row_index, col_index] = filled_value
/tmp/ipykernel_571606/3320574282.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Nuclear' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[row_index, col_index] = filled_value
/tmp/ipykernel_571606/3320574282.py:31: FutureWarning: Set


Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/MT_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/NL_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/NO_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/PL_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/PT_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/RO_installed_capacity.csv
Pro

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Managing N/A and n/e values.
</div>

In [56]:
def replace_empty_and_non_values(file_list):
  """Replaces empty strings and non-values in CSV files with zeros.

  Args:
    file_list: A list of CSV file paths.
  """

  for file_path in file_list:
    df = pd.read_csv(file_path)
    df = df.replace(['n/e', 'N/A', ''], np.nan, regex=True)
    df = df.fillna(0)
    df.to_csv(file_path, index=False)
      
replace_empty_and_non_values(total_installed_capacity_per_production_type_file_path_list)

/tmp/ipykernel_571606/1334629397.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(['n/e', 'N/A', ''], np.nan, regex=True)
/tmp/ipykernel_571606/1334629397.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(['n/e', 'N/A', ''], np.nan, regex=True)


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Changing and summing the technology generation denomination in the total_installed_capacity_per_production_type files for those ones with the Dispa-SET format.
</div>

In [57]:
# Define the replacement dictionary
replacement_dict = {
    'Fossil Peat': 'PEA',
    'Nuclear': 'NUC',
    'Fossil Hard coal': 'HRD',
    'Geothermal': 'GEO',
    'Wind Onshore': 'WIN',
    'Wind Offshore': 'WIN',
    'Fossil Brown coal/Lignite': 'LIG',
    'Hydro Run-of-river and poundage': 'WAT',
    'Hydro Water Reservoir': 'WAT',
    'Hydro Pumped Storage': 'WAT',
    'Other renewable': 'OTH',
    'Solar': 'SUN',
    'Fossil Oil shale': 'OIL',
    'Waste': 'WST',
    'Fossil Gas': 'GAS',
    'Fossil Coal-derived gas': 'GAS',
    'Fossil Oil': 'OIL',
    'Marine': 'OTH',
    'Other': 'OTH',
    'Biomass': 'BIO'
}

# Function to process each CSV file
def process_csv(file_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Replace the values in the 'Production Type' column
    df['Production Type'] = df['Production Type'].replace(replacement_dict)

    # Convert all columns except 'Production Type' to numeric, errors='coerce' will convert non-numeric data to NaN
    for column in df.columns:
        if column != 'Production Type':
            df[column] = pd.to_numeric(df[column], errors='coerce')

    # Group by 'Production Type' and sum the other columns
    grouped_df = df.groupby('Production Type', as_index=False).sum()

    # Write the updated DataFrame back to the CSV file
    grouped_df.to_csv(file_path, index=False)

    print(f"Processed file: {file_path}")

# Process each CSV file in the list
for file_path in total_installed_capacity_per_production_type_file_path_list:
    process_csv(file_path)

print("All CSV files processed successfully.")

Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/AT_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BE_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BG_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CH_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CY_installed_capacity.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CZ_installed_capacity.csv
Proc

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Creating the Final_Clean_File_Path list needed to operate with the total_installed_capacity_per_production_type_file_path_list
</div>

In [58]:
def extract_file_paths(power_plants_raw_data_sources_file_path):
  """Extracts unique file paths from a CSV file.

  Args:
    power_plants_raw_data_sources_file_path: Path to the CSV file.

  Returns:
    A list of unique file paths.
  """

  df = pd.read_csv(power_plants_raw_data_sources_file_path)
  Final_Clean_File_Path_List = df['Final_Clean_File_Path'].unique().tolist()
  Final_Clean_File_Path_List.sort()
  return Final_Clean_File_Path_List
Final_Clean_File_Path_List = extract_file_paths(power_plants_raw_data_sources_file_path)
print(Final_Clean_File_Path_List)

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/IE/20

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Adding the Solar and Wind power units needed to compensate the total installed capacity difference due to decentralized generation.
</div>

In [59]:
# Function to process the CSV files
def process_csv_files(file_path_1, file_path_2, data_year):
    # Read the CSV files into DataFrames
    df1 = pd.read_csv(file_path_1)
    df2 = pd.read_csv(file_path_2)

    # Extract the folder name from file_path_1
    folder_name = os.path.basename(os.path.dirname(file_path_1))

    # Function to calculate sum for specific fuel type
    def sum_power_capacity(df, fuel_column, capacity_column, fuel_type):
        return df[df[fuel_column] == fuel_type][capacity_column].sum()

    # Calculate sums for 'SUN'
    sun_power_capacity_1 = sum_power_capacity(df1, 'Fuel', 'PowerCapacity', 'SUN')
    sun_power_capacity_2 = sum_power_capacity(df2, 'Production Type', str(data_year), 'SUN')

    # Check and update for 'SUN'
    if sun_power_capacity_2 > sun_power_capacity_1:
        new_row = pd.DataFrame({'Unit': [f'Dispa-SET_Solar_{folder_name}'], 'PowerCapacity': [sun_power_capacity_2 - sun_power_capacity_1], 'Fuel': ['SUN']})
        df1 = pd.concat([df1, new_row], ignore_index=True)

    # Calculate sums for 'WIN'
    win_power_capacity_1 = sum_power_capacity(df1, 'Fuel', 'PowerCapacity', 'WIN')
    win_power_capacity_2 = sum_power_capacity(df2, 'Production Type', str(data_year), 'WIN')

    # Check and update for 'WIN'
    if win_power_capacity_2 > win_power_capacity_1:
        new_row = pd.DataFrame({'Unit': [f'Dispa-SET_WIN_{folder_name}'], 'PowerCapacity': [win_power_capacity_2 - win_power_capacity_1], 'Fuel': ['WIN']})
        df1 = pd.concat([df1, new_row], ignore_index=True)

    # Write the updated DataFrame back to the CSV file
    df1.to_csv(file_path_1, index=False)

    print(f"Processed {file_path_1} with data from {file_path_2} successfully.")

# Loop through the file paths and process each pair
for file_path_1, file_path_2 in zip(Final_Clean_File_Path_List, total_installed_capacity_per_production_type_file_path_list):
    process_csv_files(file_path_1, file_path_2, data_year)

Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv with data from /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/AT_installed_capacity.csv successfully.
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv with data from /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BE_installed_capacity.csv successfully.
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv with data from /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BG_installed_capacity.csv successfully.
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv with data from /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CH_installed_capacity.csv successfully.
Proc

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Adding the rest of the technologies power units needed (according their fuel type) to compensate the total installed capacity difference due to decentralized generation.
</div>

In [60]:
def process_csv_files(file_path_1, file_path_2, data_year):
  """
  Processes CSV files, calculates differences in power capacity for specified fuel types, and updates the first DataFrame accordingly.

  Args:
      file_path_1: The path to the first CSV file.
      file_path_2: The path to the second CSV file.
      data_year: The data year.
  """

  # Read the CSV files into DataFrames
  df1 = pd.read_csv(file_path_1)
  df2 = pd.read_csv(file_path_2)

  # Extract the folder name from file_path_1
  folder_name = os.path.basename(os.path.dirname(file_path_1))

  # Function to calculate sum for specific fuel type
  def sum_power_capacity(df, fuel_column, capacity_column, fuel_type):
      return df[df[fuel_column] == fuel_type][capacity_column].sum()

  # Define the list of fuel types
  fuel_types = ['BIO', 'GAS', 'GEO', 'HRD', 'LIG', 'NUC', 'OIL', 'OTH', 'PEA', 'WAT', 'WST']

  # Iterate over each fuel type
  for fuel_type in fuel_types:
      # Calculate sums for the current fuel type
      fuel_power_capacity_1 = sum_power_capacity(df1, 'Fuel', 'PowerCapacity', fuel_type)
      fuel_power_capacity_2 = sum_power_capacity(df2, 'Production Type', str(data_year), fuel_type)

      # Check and update for the current fuel type
      if fuel_power_capacity_2 > fuel_power_capacity_1:
          new_row = pd.DataFrame({'Unit': [f'Dispa-SET_{fuel_type}_{folder_name}'], 'PowerCapacity': [fuel_power_capacity_2 - fuel_power_capacity_1], 'Fuel': [fuel_type]})
          df1 = pd.concat([df1, new_row], ignore_index=True)

  # Write the updated DataFrame back to the CSV file
  df1.to_csv(file_path_1, index=False)

  print(f"Processed {file_path_1} with data from {file_path_2} successfully.")

# Loop through the file paths and process each pair
for file_path_1, file_path_2 in zip(Final_Clean_File_Path_List, total_installed_capacity_per_production_type_file_path_list):
  process_csv_files(file_path_1, file_path_2, data_year)

Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv with data from /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/AT_installed_capacity.csv successfully.
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv with data from /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BE_installed_capacity.csv successfully.
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv with data from /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/BG_installed_capacity.csv successfully.
Processed /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv with data from /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Total_Installed_Capacity_per_Production_Type/CH_installed_capacity.csv successfully.
Proc

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Filling the 'Technology', 'Zone' and 'Nunits' fileds of the added units.
</div>

In [61]:
def process_csv_file(file_path):
  """
  Processes a CSV file, filling missing values for Technology and Zone based on Fuel, and assigning default Nunits.

  Args:
      file_path: The path to the CSV file.
  """

  # Extract the folder name from the file path
  folder_name = os.path.basename(os.path.dirname(file_path))

  # Read the CSV file into a DataFrame
  df = pd.read_csv(file_path)

  # Ensure the 'Technology' and 'Zone' columns are of string type
  df['Technology'] = df['Technology'].astype(str)
  df['Zone'] = df['Zone'].astype(str)

  # Define a dictionary mapping Fuels to Technologies
  fuel_tech_mapping = {
      'BIO': 'STUR',
      'GAS': 'COMC',
      'GEO': 'STUR',
      'HRD': 'STUR',
      'LIG': 'STUR',
      'NUC': 'STUR',
      'OIL': 'STUR',
      'OTH': 'STUR',
      'PEA': 'STUR',
      'WAT': 'HROR',
      'WST': 'STUR',
      'SUN': 'PHOT',
      'WIN': 'WTOF'
  }

  # Process the DataFrame
  for index, row in df.iterrows():
    # Fill missing Technology based on Fuel
    if pd.isna(row['Technology']) or row['Technology'] == 'nan':
      technology = fuel_tech_mapping.get(row['Fuel'], None)
      if technology:
        df.at[index, 'Technology'] = technology

    # Fill missing Zone with folder name
    if pd.isna(row['Zone']) or row['Zone'] == 'nan':
      df.at[index, 'Zone'] = folder_name

    # Fill missing Nunits with 1
    if pd.isna(row['Nunits']):
      df.at[index, 'Nunits'] = 1

  # Save the updated DataFrame back to the CSV file
  df.to_csv(file_path, index=False)

# Loop through the file paths and process each file
for file_path in Final_Clean_File_Path_List:
  process_csv_file(file_path)

print("All CSV files have been processed and updated successfully.")

All CSV files have been processed and updated successfully.


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.12. CHPType, CHPPowerToHeat, CHPPowerLossFactor and CHPMaxHeat Fields Filter
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Re-naming the current value of all the fields of CHPType column.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Since the fuel sources names of the power units may not be homogeneous among all data sources. The current value of CHPType field is compared with the CHP equivalence dictionary database to obtain the one corresponding to the Dispa-SET format.
</div>
<div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
    This part reads for each unit the current value of the column called "CHPMaxHeat" of power_plants_clean_data file. If the unit has any value on this field,  checks out its respective value in the column CHPType, and compares wiht the table power_plants_all_data_equivalent_CHPTypes_file, if there is no coincydense, copies the row to the power_plants_all_data_not_defined_units_file leaving this field empty, but keeping the information of the other related fields.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    This process is done for each zone.
</div>

In [62]:
from Data_Processing_Functions import update_chp_types

# Iterate over each pair of files
for clean_data_file_path, not_defined_units_file_path in zip(power_plants_clean_data_file_list, power_plants_all_data_not_defined_units_file_list):
    # Call the function with appropriate arguments
    update_chp_types(clean_data_file_path, power_plants_all_data_equivalent_CHPTypes_file_path, not_defined_units_file_path)

CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.
CHP types updated successfully.


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.13. Power Capacity Field Filter
</div>
<div style="text-align: jusitfy; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Erasing all the units with the Power Capacity field empty or with zero value.
         <br>
    This process is done for each zone.
</div>

In [63]:
from Data_Processing_Functions import remove_zero_power_capacity

# Iterate over each pair of files
for clean_data_file_path, not_defined_units_file_path in zip(power_plants_clean_data_file_list, power_plants_all_data_not_defined_units_file_list):
    # Call the function with appropriate arguments
    remove_zero_power_capacity(clean_data_file_path, not_defined_units_file_path)

Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power capacity rows removed successfully.
Zero power ca

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.14. Efficiency Field Fullfilling
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Finding the closest efficiency value for all the units with the Efficiency field empty or with zero value.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    The Efficiency technical feauture is taken from the already agregated data base EU_Power_Units_Technical_Features.csv using the notebook: EU_Power_Plant_Technical_Data_Base_Gathering.ipynb and is os sellected in base on the Technology, the Fuel and the PowerCapacity features.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    This process is done for each zone.
</div>

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [64]:
power_plants_clean_data_file_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

In [65]:
EU_Power_Units_Technical_Features_file_path

'/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/Dictionaries/EU_Power_Units_Technical_Features.csv'

In [66]:
# Import the function from the external script
from Data_Processing_Functions import copy_technical_values

# Define the paths and variables
#EU_Power_Units_Technical_Features_file_path = '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Technical_Features.csv'
#power_plants_clean_data_file_list = [
#    '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/2020_power_plants_raw_data_sources_20240407_172323/2020.csv',
#    '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2020.csv',
#    '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2020.csv',
#    '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2020.csv'
#]
common_columns = ['PowerCapacity', 'Technology', 'Fuel']
column_to_copy = 'Efficiency'

copy_technical_values(EU_Power_Units_Technical_Features_file_path, power_plants_clean_data_file_list, common_columns, column_to_copy)

Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ra

Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ra

Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv


...

Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/PT/2016.csv


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')


Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/RO/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/SE/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/SI/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/SK/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/UK/2016.csv


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ra

Efficiency Field values copied successfully.


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Filling with a value of 1 the efficiency technical feature for all the units with the Efficiency field empty or with zero value.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Additionally all this units found with any efficiency value, will be copied to the corresponding no denined units file.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    This process is done for each zone.
</div>

In [67]:
from Data_Processing_Functions import fill_empty_values_with_specified

# Define the columns to be filled
Column = 'Efficiency'
Value = '1'  # Change this to the desired value

# Iterate over each pair of files
for input_file, output_file in zip(power_plants_clean_data_file_list, power_plants_all_data_not_defined_units_file_list):
    # Call the function for the current pair of files
    fill_empty_values_with_specified(input_file, output_file, Column, Value)

No empty 'Efficiency' or value 0 cells found in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv'.
No empty 'Efficiency' or value 0 cells found in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv'.
No empty 'Efficiency' or value 0 cells found in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv'.
No empty 'Efficiency' or value 0 cells found in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv'.
No empty 'Efficiency' or value 0 cells found in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv'.
No empty 'Efficiency' or value 0 cells found in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv'.
No empty 'Efficiency' or value 0 cells found in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv'.
No empty 'Efficiency' or value 0 cells found in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv'.
No empty 'Efficiency' or value 0 cells found in '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv'.
N

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    7.14.  MinDownTime | MinDownTime | RampUpRate | RampDownRate | StartUpCost | NoLoadCost_pu | RampingCost | PartLoadMin | MinEfficiency | StartUpTime | CO2Intensity Fields Fulfilling
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Finding the closest value for the next technical features:
 </div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    MinUpTime, MinDownTime, MinDownTime, RampUpRate, RampDownRate, StartUpCost, NoLoadCost_pu, RampingCost, PartLoadMin, MinEfficiency, StartUpTime, CO2Intensity.
 </div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    This is done for all the units which corresponding field value is empty or with zero.
    </div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    All the technical feautures are taken from the already agregated data base EU_Power_Units_Technical_Features.csv using the notebook: EU_Power_Plant_Technical_Data_Base_Gathering.ipynb and are sellected in base on the Technology, the Fuel and the PowerCapacity features.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    This process is done for each zone.
</div>

In [68]:
columns_to_copy = [
    'MinUpTime',
    'MinDownTime',
    'RampUpRate',
    'RampDownRate',
    'StartUpCost',
    'NoLoadCost_pu',
    'RampingCost',
    'PartLoadMin',
    'MinEfficiency',
    'StartUpTime',
    'CO2Intensity'
]
common_columns = ['PowerCapacity', 'Technology', 'Fuel']

In [69]:
for column_to_copy in columns_to_copy:
    copy_technical_values(EU_Power_Units_Technical_Features_file_path, power_plants_clean_data_file_list, common_columns, column_to_copy)

Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ra

Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ra

Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv


...

Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/PT/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/RO/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/SE/2016.csv


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ra

Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/SI/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/SK/2016.csv
Processing target file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/UK/2016.csv


/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ray/Dispa-SET_Unleash/scripts/Unleash_Raw_Data_Processing/Data_Processing_Functions.py:584: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_fuel_filtered['PowerCapacity'] = pd.to_numeric(first_fuel_filtered['PowerCapacity'], errors='coerce')
/home/ra

CO2Intensity Field values copied successfully.


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    8. Zone Final Classification
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Filtering by Zone all the formating files resulting from the downloads files set by the "General" key at the begining of the process.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Each row of the files are copied according their zone clasification in their corresponding folder path. 
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    8.1. Zone Field Homogenization
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Eliminating the space characters from the Zone fileds to avoid errors in the zone classifiaction process.
    <br>
    This process is done for each zone.
</div>

In [70]:
# List of file paths
#power_plants_clean_data_file_list = [
#    '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/2020_power_plants_raw_data_sources_20240407_201458/2020.csv',
#    '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2020.csv',
#    '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2020.csv',
#    '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2020.csv'
#]

# Iterate over each file
for file_path in power_plants_clean_data_file_list:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Trim values in the 'Zone' column
    if 'Zone' in df.columns:
        df['Zone'] = df['Zone'].str.strip()

        # Write the modified DataFrame back to the CSV file
        df.to_csv(file_path, index=False)

        print(f"Trimmed 'Zone' column values in {file_path}")
    else:
        print(f"'Zone' column not found in {file_path}")


Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv
Trimmed 'Zone' column values in /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Filtering by Zone all the formating files resulting from the downloads files set by the "General" key at the begining of the process.
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Each row of the files are copied according their zone clasification in their corresponding folder path. 
</div>

In [71]:
# File paths
#power_plants_raw_data_sources_file_path = '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/2020_power_plants_raw_data_sources_20240407_191506/2020_power_plants_raw_data_sources_20240407_191506.csv'

# Dictionary of created zones
#created_zones = {'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE',
#                 'DK': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK',
#                 'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH',
#                'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE'}

# Read the first file
first_df = pd.read_csv(power_plants_raw_data_sources_file_path)

# Filter rows with Zone as "General"
general_zones_df = first_df[first_df['Zone'] == 'General']

# Iterate over rows with Zone as "General"
for index, row in general_zones_df.iterrows():
    # Get the path of the corresponding file
    file_path = row['Final_Clean_File_Path']
    
    # Read the second CSV file
    second_df = pd.read_csv(file_path)
    
    # Iterate over zone names
    for zone in zone_names:
        # Filter rows in the second file with matching Zone
        zone_rows = second_df[second_df['Zone'] == zone]
        
        # Check if there are matching rows
        if not zone_rows.empty:
            # Get the corresponding zone path
            zone_path = created_zones.get(zone)
            
            # Create the directory if it does not exist
            if not os.path.exists(zone_path):
                os.makedirs(zone_path)
            
            # Get the file name
            file_name = os.path.basename(file_path)
            zone_file_path = os.path.join(zone_path, file_name)
            
            # Copy the rows to the corresponding zone file
            if not os.path.exists(zone_file_path):
                zone_rows.to_csv(zone_file_path, index=False)
            else:
                zone_rows.to_csv(zone_file_path, mode='a', index=False, header=False)
            
            # Remove the copied rows from the original DataFrame
            second_df.drop(zone_rows.index, inplace=True)
            
            # Print message
            print(f"Copied rows to {zone_file_path} for zone {zone}.")
    
    # Write the updated DataFrame back to the second file
    second_df.to_csv(file_path, index=False)
    print(f"Original file {file_path} updated after row deletion.")

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    9. Zone Clustering
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Grouping power units according the location, the company or the name. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    This part is set to make a cluster of all the units that have the same Latitud and Longitud location (if the data is provided)
     <br>
    However, there is the posibility to cluster the units by the name for all the units that has the same name or by the company (if the data is also provided). 
</div>
<div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman;color:skyblue">
    To do it, just Uncomment the corresponding line of the following lines of the code.
</div>
<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;">
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    9.1. Lat/Lon Column Creation
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Joining the filed of the columns Lat and Lon into a new column under the name Lat/Lon to clustering purposes.
</div>

In [72]:
#created_zones = {
#    'DE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE',
#    'DK': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK',
#    'CH': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH',
#    'BE': '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE'
#}

#data_year = '2020'

power_plants_clean_data_file_path_list = []

# Extract and format paths
for zone, path in created_zones.items():
  full_path = f"{path}/{data_year}.csv"
  power_plants_clean_data_file_path_list.append(full_path)

# Print the list of paths
print(power_plants_clean_data_file_path_list)

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv', '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/IE/20

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [73]:
power_plants_clean_data_file_path_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

In [74]:
for file_path in power_plants_clean_data_file_path_list:
  # Read the CSV file
  df = pd.read_csv(file_path)

  # Check if "Lat" and "Lon" columns exist
  if 'Lat' in df.columns and 'Lon' in df.columns:
    # Create a new column named "Lat+Lon" with empty string as default
    df['Lat/Lon'] = np.where((df['Lat'].notna()) & (df['Lon'].notna()), df['Lat'].astype(str) + "/" + df['Lon'].astype(str), '')

    # Save the modified DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
  else:
    print(f"Warning: 'Lat' or 'Lon' column not found in {file_path}")

print("Lat/Lon column added to files (if columns exist).")

Lat/Lon column added to files (if columns exist).


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    9.2. Fields Cluster
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Clustering the units with the repeated values in the Lat/Lon.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    The next code just clusters the units which Latitud and longitud repeated values under the condition that have the same Technology, Fuel and CHPType as well.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    This process is made for each zone.
    <br>
    There is the posibility to make a cluster looking for duplicate fields in the name of the units or the company (If the data exists). 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">    
For this purpouse just uncomend the corresponding lines.
</div>

In [75]:
# Column to be looked for
column_to_look_for = 'Lat/Lon'  # Change 'YourColumnNameHere' to the actual column name
#column_to_look_for = 'Unit'
#column_to_look_for = 'Company'


# Iterate over each file
for file_path in power_plants_clean_data_file_path_list:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Check if the column to look for exists
    if column_to_look_for in df.columns:
        # Iterate over each value in the specified column
        for value in df[column_to_look_for].unique():
            # Check if the field has some value
            if pd.notnull(value):
                # Filter rows where the specified column has the exact same value
                matched_rows = df[df[column_to_look_for] == value]

                # Check if there are more than one row with the same value
                if len(matched_rows) > 1:
                    # Perform additional matching based on Technology, Fuel, and CHPType
                    # You can adjust the conditions as per your requirement
                    matched_rows = matched_rows.groupby(['Technology', 'Fuel', 'CHPType']).filter(lambda x: len(x) > 1)

                    if len(matched_rows) > 1:
                        # Sum the corresponding fields
                        summed_fields = matched_rows[['PowerCapacity', 'Nunits', 'StartUpCost', 'NoLoadCost_pu',
                                                       'RampingCost', 'CHPPowerToHeat', 'CHPMaxHeat', 'STOCapacity',
                                                       'WaterWithdrawal', 'WaterConsumption']].sum()

                        # Calculate the average of the corresponding fields
                        averaged_fields = matched_rows[['Efficiency', 'MinUpTime', 'MinDownTime', 'RampUpRate',
                                                         'RampDownRate', 'PartLoadMin', 'MinEfficiency', 'StartUpTime',
                                                         'CO2Intensity', 'CHPPowerLossFactor', 'CHPMaxHeat', 'COP',
                                                         'Tnominal', 'coef_COP_a', 'coef_COP_b', 'STOSelfDischarge',
                                                         'STOMaxChargingPower', 'STOChargingEfficiency']].mean()

                        # Keep the first row and update its values with the sums and averages
                        first_row_index = matched_rows.index[0]
                        df.loc[first_row_index, ['PowerCapacity', 'Nunits', 'StartUpCost', 'NoLoadCost_pu',
                                                 'RampingCost', 'CHPPowerToHeat', 'CHPMaxHeat', 'STOCapacity',
                                                 'WaterWithdrawal', 'WaterConsumption']] = summed_fields
                        df.loc[first_row_index, ['Efficiency', 'MinUpTime', 'MinDownTime', 'RampUpRate',
                                                 'RampDownRate', 'PartLoadMin', 'MinEfficiency', 'StartUpTime',
                                                 'CO2Intensity', 'CHPPowerLossFactor', 'CHPMaxHeat', 'COP',
                                                 'Tnominal', 'coef_COP_a', 'coef_COP_b', 'STOSelfDischarge',
                                                 'STOMaxChargingPower', 'STOChargingEfficiency']] = averaged_fields

                        # Copy the matched rows to a new DataFrame
                        clustered_df = matched_rows.copy()

                        # Write the matched rows to a new CSV file
                        output_file_path = os.path.splitext(file_path)[0] + '_clustered.csv'
                        clustered_df.to_csv(output_file_path, index=False)

                        # Erase the rows from the original DataFrame
                        df.drop(matched_rows.index, inplace=True)

    # Write the modified DataFrame back to the original CSV file
    df.to_csv(file_path, index=False)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    10. Storage Capacity Data
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Filling the 'Storage Capacity' fields of each power unit of the power plant data.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
    Data from the 'JRC Hydro-power database' source is used, the same can be dowloaded from:
<div style="text-align: justify; margin-left: 4.0em; font-weight: normal; font-size: 12px; font-family: Times New Roman;color:skyblue">
    <a href="https://zenodo.org/records/5215920" target="_blank">https://zenodo.org/records/5215920</a>
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    The source data is found in the following folder into the Dispa-SET local directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: normal; font-size: 13px; font-family: Times New Roman;color:skyblue">
    /local/ditectory/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Hydro_Units
</div>

In [76]:
additional_path_2 = "/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Hydro_Units/jrc-hydro-power-plant-database.csv"
hydro_units_raw_data_file_path = dispaSET_unleash_folder_path + additional_path_2
print (f"hydro_units_raw_data_file_path: {hydro_units_raw_data_file_path}")

# Read the hydro units CSV file
hydro_units_df = pd.read_csv(hydro_units_raw_data_file_path)

for file_path_1 in power_plants_clean_data_file_path_list:
    # Read the current CSV file
    file_1_df = pd.read_csv(file_path_1)

    # Extract unique zones from the current file
    unique_zones = file_1_df['Zone'].unique()

    # Filter hydro_units_df by country_code
    filtered_hydro_units_df = hydro_units_df[hydro_units_df['country_code'].isin(unique_zones)]

    # Further filter by type 'HDAM' or 'HPHS'
    filtered_hydro_units_df = filtered_hydro_units_df[filtered_hydro_units_df['type'].isin(['HDAM', 'HPHS'])]

    # Filter file_1_df by Technology 'HDAM' or 'HPHS'
    filtered_file_1_df = file_1_df[file_1_df['Technology'].isin(['HDAM', 'HPHS'])]

    # Create a mapping from installed_capacity_MW to storage_capacity_MWh
    capacity_to_storage = filtered_hydro_units_df.set_index('installed_capacity_MW')['storage_capacity_MWh'].to_dict()

    # Define a function to update 'STOCapacity'
    def update_storage_capacity(row):
        power_capacity = row['PowerCapacity']
        return capacity_to_storage.get(power_capacity, None)

    # Apply the update function
    filtered_file_1_df['STOCapacity'] = filtered_file_1_df.apply(update_storage_capacity, axis=1)

    # Identify rows with no direct match
    no_match_df = filtered_file_1_df[filtered_file_1_df['STOCapacity'].isnull()]

    # Calculate the total storage capacity from unmatched hydro units
    unmatched_hydro_units_df = filtered_hydro_units_df[
        ~filtered_hydro_units_df['installed_capacity_MW'].isin(filtered_file_1_df['PowerCapacity'])
    ]
    total_unmatched_storage_capacity = unmatched_hydro_units_df['storage_capacity_MWh'].sum()

    # Calculate the proportional storage capacity for unmatched rows
    if not no_match_df.empty:
        total_power_capacity = no_match_df['PowerCapacity'].sum()
        if total_power_capacity > 0:
            no_match_df['STOCapacity'] = (no_match_df['PowerCapacity'] / total_power_capacity) * total_unmatched_storage_capacity

        # Update the filtered file DataFrame
        filtered_file_1_df.update(no_match_df)

    # Update the original DataFrame
    file_1_df.update(filtered_file_1_df)

    # Save the updated original DataFrame to the same CSV file
    file_1_df.to_csv(file_path_1, index=False)

hydro_units_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EU_Power_Units_Raw_Data_Source/Hydro_Units/jrc-hydro-power-plant-database.csv


/tmp/ipykernel_571606/2230097895.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_file_1_df['STOCapacity'] = filtered_file_1_df.apply(update_storage_capacity, axis=1)
/tmp/ipykernel_571606/2230097895.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_match_df['STOCapacity'] = (no_match_df['PowerCapacity'] / total_power_capacity) * total_unmatched_storage_capacity
/tmp/ipykernel_571606/2230097895.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
    11. Self-Discharge Rate | Maximum Charging Power | Charging efficiency  Data 
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Filling the 'STOSelfDischarge', 'STOMaxChargingPower', 'STOChargingEfficiency' fields of each .corresponding Hydro unit of the power plant data.
</div>

In [77]:
def process_csv_files(power_plants_raw_data_sources_file_path):
    """
    Processes CSV files, modifying values for HPHS and HDAM technologies.

    Args:
        power_plants_raw_data_sources_file_path: The path to the CSV file containing file paths.
    """

    df = pd.read_csv(power_plants_raw_data_sources_file_path)

    for index, row in df.iterrows():
        file_path = row['Final_Clean_File_Path']
        if pd.notnull(file_path):
            process_csv_file(file_path)

def process_csv_file(file_path):
    """
    Processes a CSV file, modifying values for HPHS and HDAM technologies.

    Args:
        file_path: The path to the CSV file.
    """

    df = pd.read_csv(file_path)

    # Process HPHS rows
    df.loc[df['Technology'] == 'HPHS', 'STOSelfDischarge'] = 0
    df.loc[df['Technology'] == 'HPHS', 'STOMaxChargingPower'] = df.loc[df['Technology'] == 'HPHS', 'PowerCapacity']
    df.loc[df['Technology'] == 'HPHS', 'STOChargingEfficiency'] = df.loc[df['Technology'] == 'HPHS', 'Efficiency']

    # Process HDAM rows
    df.loc[df['Technology'] == 'HDAM', 'STOSelfDischarge'] = 0
    df.loc[df['Technology'] == 'HDAM', 'STOMaxChargingPower'] = 0
    df.loc[df['Technology'] == 'HDAM', 'STOChargingEfficiency'] = 0

    df.to_csv(file_path, index=False)
    print(f"Processed file: {file_path}")

process_csv_files(power_plants_raw_data_sources_file_path)

Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv
Processed fi

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [78]:
power_plants_clean_data_file_path_list

['/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EE/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/EL/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/ES/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FI/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/FR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HR/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/HU/2016.csv',
 '/home/ray/Dispa-SET_Unleash/RawData/Po

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;color:skyblue">
11. Copying Power Plants Formatted Data
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Moving the already clean and formatted data of the power units to the main Dispa-SET data base directory.
     <br>
    <div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The data base directory has to be identified.
</div>

In [79]:
additional_path_4 = "/Database/PowerPlants/"

# Construct the power_plants_raw_data_folder_path variable
power_plants_data_base_folder_path = dispaSET_unleash_folder_path + additional_path_4

In [80]:
# Iterate over each zone in the list
for zone in zone_names:
    # Define the source file path
    source_file_path = os.path.join(power_plants_raw_data_folder_path, zone, f"{data_year}.csv")
    
    # Define the destination folder path
    destination_folder_path = os.path.join(power_plants_data_base_folder_path, zone)
    
    # Define the destination file path
    destination_file_path = os.path.join(destination_folder_path, f"{data_year}.csv")
    
    # Check if the source file exists
    if os.path.exists(source_file_path):
        # Ensure the destination folder exists
        os.makedirs(destination_folder_path, exist_ok=True)
        
        # Copy the file
        shutil.copyfile(source_file_path, destination_file_path)
        print(f"Copied {source_file_path} to {destination_file_path}")
    else:
        print(f"Source file does not exist: {source_file_path}")

print("Task completed.")

Copied /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/AT/2016.csv to /home/ray/Dispa-SET_Unleash/Database/PowerPlants/AT/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BE/2016.csv to /home/ray/Dispa-SET_Unleash/Database/PowerPlants/BE/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/BG/2016.csv to /home/ray/Dispa-SET_Unleash/Database/PowerPlants/BG/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CH/2016.csv to /home/ray/Dispa-SET_Unleash/Database/PowerPlants/CH/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CY/2016.csv to /home/ray/Dispa-SET_Unleash/Database/PowerPlants/CY/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/CZ/2016.csv to /home/ray/Dispa-SET_Unleash/Database/PowerPlants/CZ/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2016.csv to /home/ray/Dispa-SET_Unleash/Database/PowerPlants/DE/2016.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DK/2016.csv to /home/ray/Dis

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
12. Power Plants Folder Back Up
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Once all the formating process was done the Power Plants Folder is restored to its defoult state.
</div>

In [81]:
if os.path.exists(power_plants_raw_data_folder_path):
    shutil.rmtree(power_plants_raw_data_folder_path)  # Remove the current directory
shutil.copytree(backup_folder_path, power_plants_raw_data_folder_path)

print(f"Directory restored to original state from {backup_folder_path}")

Directory restored to original state from /home/ray/Dispa-SET_Unleash/RawData/PowerPlants_backup/


In [82]:
shutil.rmtree(backup_folder_path)
print(f"Backup folder {backup_folder_path} deleted successfully.")

Backup folder /home/ray/Dispa-SET_Unleash/RawData/PowerPlants_backup/ deleted successfully.
